## This kernel is now just bringing together our features and 'log normalizing' them
- If there is a 'real' meaning of 'log normalization', this isn't it
- This is my practical method for **compressing features with wide spreads across both negative and positive values**
- I have no theoretical justification however empirically it has improved the performance of my models


## standardScaler and .fillna weren't working reliably for me
- My cleanDf method has five stages
- ensureNoNanOrInf
- convertTFToInt (this method should be using the dataType of the input rather than feature naming conventions)
- convertAllToLogBase
- removeExtremeValues
- featureScaleAllExcept


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir('../input'))
print(os.listdir("../input/writefeaturetablefromsmotedartset"))
print(os.listdir('../input/normalizesomethingdifferentfeatures'))
# Any results you write to the current directory are saved as output.

## Load and merge the training data
- trainingDartDf is from Chai-Ta Tsai's kernel
- trainingJimsDf is from my somethingDifferent kernel


In [ ]:
#Here is a change from the script
#training features
trainingDartDf=pd.read_csv('../input/writefeaturetablefromsmotedartset/trainingFeatures1039.csv')
trainingJimsDf=pd.read_csv('../input/normalizesomethingdifferentfeatures/traindfNormal.csv')
if 'Unnamed: 0' in trainingDartDf.columns:
    trainingDartDf=trainingDartDf.drop('Unnamed: 0', axis=1)
print(trainingDartDf.shape)
#trainingDartDf.head()
columnsToAdd=['outlierScore', 'hipd', 'lipd', 'highEnergy_transitory_1.0_TF',
          'highEnergy_transitory_1.5_TF', 'lowEnergy_transitory_1.0_TF', 
          'lowEnergy_transitory_1.5_TF']

for column in columnsToAdd:
    trainingDartDf.loc[:,column]=trainingJimsDf.loc[:,column]

traindf=trainingDartDf

#from the 1.052 kernel
del traindf['hostgal_specz']
del traindf['ra'], traindf['decl'], traindf['gal_l'], traindf['gal_b']
del traindf['ddf']


print(traindf.shape)
traindf.head()

## My neural net model did better when I treated the intragalactic and extragalactic objects differently

In [ ]:
import copy

igTrain=copy.deepcopy(traindf)[traindf['distmod'].isna()]

egTrain=copy.deepcopy(traindf)[~traindf['distmod'].isna()]

In [ ]:
print(igTrain.shape)
igTrain.describe()

In [ ]:
print(egTrain.shape)

## I saw an improvement using the cleanupDf(traindf, testdf) method at the bottom here
- Because train and test are quite different for this project it was important that they be processed the same way
- Imagine if I trained on log(feat) but cleanUp(df) didn't transform test[feat].
- This normalization scheme means that the test data will not necessarily be between 0 and 1

In [ ]:
#you would really think this would be unnecessary.
#however I kept getting nan errors even after df=df.fillna(0)

def ensureNoNanOrInf(df):

    df=df.round(5)
    df.replace(np.inf, 9999, inplace=True)
    for cindex in df.columns:
        dropit=df[cindex].isna().sum()
        #print(dropit)
        finite=np.isfinite(df[cindex]).sum()
        if finite != df.shape[0]:
            print(finite)
        if dropit>0:
            #full_train=full_train.drop(cindex,axis=1)
            print(cindex + ' has ' + str(dropit) + ' nans')
            df[cindex].fillna(0,inplace=True)
    print(df.shape)
    return df

def convertTFToInt(df):
    
    for cindex in df.columns:
        
        if '_TF' in cindex:
            #print(cindex)
            df[cindex]=df[cindex].astype(int)
            
        if '_TF_' in cindex:
            #print(cindex)
            df[cindex]=df[cindex].astype(int)
            
    return df

#I used this because most of the data had a very wide spread
#I have another method that compares kurtosis and skew of feat vs ln(feat) to decide whether to transform it
#Values between -1 and 1 become 0 using this method
#maxFrac is a threshold where if too much of the data is between -1 and 1 then it won't be transformed

def convertAllToLogBase(df, testdf, maxFrac=0.05, excludeLogCols=['ra', 'decl', 'gal_l', 'gal_b', 'hostgal_photoz', 'target',
                                            'hostgal_photoz_err', 'distmod', 'outlierScore', 'object_id']):
    
    #the intent here is to find the magnitude and the directionality of the flux
    #because of weird behavior between -1 and 1 we'll treat that as 0
    #then the sign is separated from the magnitude using the filters below
    #later we'll feature scale
    dfSize=df.shape[0]
    
        
    for cindex in df.columns:
        
        if (cindex not in excludeLogCols) & (len(df.loc[:,cindex].unique()) > 2):
            
            #zeroFilter=df.loc[:,cindex]==0 (stays zero)
            zeroFilter=((df.loc[:,cindex]>-1) & (df.loc[:,cindex]<1))
            zeroFilterTest=((testdf.loc[:,cindex]>-1) & (testdf.loc[:,cindex]<1))
            if ((zeroFilter.sum() / dfSize) <= maxFrac):

                negFilter=df.loc[:,cindex]<=-1
                negFilterTest=testdf.loc[:,cindex]<=-1
                
                posFilter=df.loc[:,cindex]>=1
                posFilterTest=testdf.loc[:,cindex]>=1
                
                df.loc[zeroFilter,cindex]=0
                df.loc[negFilter,cindex]=-1.0*np.log(-1.0*df.loc[negFilter,cindex])
                df.loc[posFilter,cindex]=np.log(df.loc[posFilter,cindex])
                
                testdf.loc[zeroFilterTest,cindex]=0
                testdf.loc[negFilterTest,cindex]=-1.0*np.log(-1.0*testdf.loc[negFilterTest,cindex])
                testdf.loc[posFilterTest,cindex]=np.log(testdf.loc[posFilterTest,cindex])
                
            else:
                print('feature ' + str(cindex) + ' not log normalized because it has too many values near zero')
                        
            
    return df, testdf

#5.88 sigma was chosen based on 500,000,000 rows of data in the PLAsTiCC test set and the z-statistic
def removeExtremeValues(df, testdf, maxSig=5.88):
    
    for cindex in df.columns:
        if cindex not in ['object_id', 'target']:
            med=np.median(df[cindex])
            sig=np.std(df[cindex])
            minVal=med-maxSig*sig
            maxVal=med+maxSig*sig
            highFilter=(df.loc[:,cindex]>maxVal) | (df.loc[:,cindex] == np.inf)
            highFilterTest=(testdf.loc[:,cindex]>maxVal) | (testdf.loc[:,cindex] == np.inf)
            
            lowFilter=(df.loc[:,cindex]<minVal) | (df.loc[:,cindex] == -np.inf)
            lowFilterTest=(testdf.loc[:,cindex]<minVal) | (testdf.loc[:,cindex] == -np.inf)
            
            df.loc[lowFilter,cindex]=minVal
            testdf.loc[lowFilterTest,cindex]=minVal
            
            df.loc[highFilter,cindex]=maxVal
            testdf.loc[highFilterTest,cindex]=maxVal
    
    return df, testdf

def featureScaleAllExcept(df, testdf, targCol='target'):
    
    for cindex in df.columns:
        if (cindex != targCol) & (cindex !='object_id'):
            minval=np.min(df.loc[:,cindex])
            maxval=np.max(df.loc[:,cindex])
            theRange=(maxval-minval)
            if theRange==0:
                #this feature contains no information
                df=df.drop(cindex,axis=1)
                testdf=testdf.drop(cindex, axis=1)
                print('dropped ' + str(cindex) + ' from dataFrame because training set has no useful information about this feature')
                
            elif type(df[cindex])==bool:
                print(str(cindex) + ' is a boolean, doing nothing')
                
            elif theRange==1:
                print('the range for ' + str(cindex) + ' is already 1, doing nothing')
                
            else:
                #feature scale
                df.loc[:,cindex]=(df.loc[:,cindex]-minval)/theRange
                testdf.loc[:,cindex]=(testdf.loc[:,cindex]-minval)/theRange
                
    return df, testdf

def cleanupDf(traindf, testdf):
    
    traindf=ensureNoNanOrInf(traindf)
    testdf=ensureNoNanOrInf(testdf)
    
    traindf=convertTFToInt(traindf)
    testdf=convertTFToInt(testdf)
    
    traindf, testdf=convertAllToLogBase(traindf, testdf)
    
    traindf, testdf=removeExtremeValues(traindf, testdf, maxSig=5.88)
    traindf, testdf=featureScaleAllExcept(traindf, testdf)
    
    traindf=traindf.round(5)
    testdf=testdf.round(5)
    
    return traindf, testdf





#traindf, forceCols=cleanupDf(traindf)
#traindf.describe()

## Load the test data
- The purpose of forceCols is to ensure that the test set receives the same treatment as the training set
- Imagine you set maxFrac at 0.05 and the training set had 4% near zero but the test set had 6% near zero.  The two sets would get a different treatment

In [ ]:
#test features
testDartDf=pd.read_csv('../input/writefeaturetablefromsmotedartset/feat_0.648970_2018-11-23-09-00.csv')
testJimsDf=pd.read_csv('../input/normalizesomethingdifferentfeatures/testdfNormal.csv')

if 'Unnamed: 0' in testDartDf.columns:
    testDartDf=testDartDf.drop('Unnamed: 0', axis=1)
print(testDartDf.shape)
testDartDf.head()

for column in columnsToAdd:
    testDartDf.loc[:,column]=testJimsDf.loc[:,column]

testdf=testDartDf

#from the 1.052 kernel
del testdf['hostgal_specz']
del testdf['ra'], testdf['decl'], testdf['gal_l'], testdf['gal_b']
del testdf['ddf']

testdf.shape
igTest=copy.deepcopy(testdf)[testdf['distmod'].isna() | testdf['distmod']==0]
print(igTest.shape)
egTest=copy.deepcopy(testdf)[~testdf['distmod'].isna() & testdf['distmod']!=0]
print(egTest.shape)


In [ ]:
igTrain, igTest = cleanupDf(igTrain, igTest)
egTrain, egTest = cleanupDf(egTrain, egTest)

#testdf, forceCols=cleanupDf(testdf, forceCols)
#print(testdf.shape)
#testdf.describe()
#testdf.to_csv('logNormalTest.csv', index=False)

In [ ]:
igTrain.describe()

In [ ]:
egTrain.describe()

In [ ]:
igTest.describe()

In [ ]:
egTest.describe()

In [ ]:
igTrain.to_csv('igTrain.csv', index=False)
igTest.to_csv('igTest.csv', index=False)
egTrain.to_csv('egTrain.csv', index=False)
egTest.to_csv('egTest.csv', index=False)


## My LGBM model didn't do any better with split data and it was easier to use unsplit data
- So this kernel saves both split and unsplit data

In [ ]:
for cindex in egTest.columns:
    if cindex not in igTest.columns:
        igTest[cindex]=0
        
for cindex in egTrain.columns:
    if cindex not in igTrain.columns:
        igTrain[cindex]=0
        
testdf=pd.concat([igTest, egTest], sort=True)

traindf=pd.concat([igTrain, egTrain], sort=True)

print(testdf.shape)
print(traindf.shape)

traindf.describe()
traindf.to_csv('fullTrain.csv', index=False)
testdf.to_csv('fullTest.csv', index=False)